In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.initializers import glorot_uniform, random_uniform
from sklearn.metrics import confusion_matrix
import itertools
import cv2
import glob

# Configuration
dataset_url = r'C:\Users\abudh\Desktop\CropWatch\EuroSAT\2750'
test_folder = r'C:\Users\abudh\Desktop\CropWatch\Test'
filtered_image_folder = r'C:\Users\abudh\Desktop\CropWatch\Filtered_Image'
model_path = 'resnet50_model.h5'
batch_size = 32
img_height = 64
img_width = 64
validation_split = 0.2
rescale = 1.0 / 255
epoch = 10
n = 3  # Number of rows and columns to splice the image into
vegetation_classes = ["AnnualCrop", "Forest", "HerbaceousVegetation", "Pasture", "PermanentCrop"]

# Identity block for ResNet
def identity_block(X, f, filters, training=True, initializer=random_uniform):
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(filters=F1, kernel_size=1, strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    return X

# Convolutional block for ResNet
def convolutional_block(X, f, filters, s=2, training=True, initializer=glorot_uniform):
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(filters=F1, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

# ResNet model definition
def ResNet50(input_shape=(64, 64, 3), classes=6):
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer='glorot_uniform')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])
    X = AveragePooling2D(pool_size=(2, 2))(X)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer='glorot_uniform')(X)
    model = tf.keras.models.Model(inputs=X_input, outputs=X, name='ResNet50')
    return model

# Plot training history
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16.53, 11.69))
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_xlabel('epoch')
    ax1.set_ylabel('accuracy')
    ax1.set_title('Accuracy over epochs')
    ax1.legend(['Train', 'Validation'], loc='upper left')
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_xlabel('epoch')
    ax2.set_ylabel('loss')
    ax2.set_title('Loss over epochs')
    ax2.legend(['Train', 'Validation'], loc="upper right")
    plt.show()

# Prepare dataset
def prepare_data(dataset_url, img_height, img_width, batch_size, validation_split, rescale):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=validation_split, rescale=rescale)
    train_dataset = datagen.flow_from_directory(directory=dataset_url, target_size=(img_height, img_width), batch_size=batch_size, subset="training", class_mode='categorical')
    test_dataset = datagen.flow_from_directory(directory=dataset_url, target_size=(img_height, img_width), batch_size=batch_size, subset="validation", class_mode='categorical')
    return train_dataset, test_dataset, list(train_dataset.class_indices.keys())

# Train model
def train_model(train_dataset, test_dataset, class_names, epochs=epoch):
    model = ResNet50(input_shape=(img_height, img_width, 3), classes=len(class_names))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs)
    model.save(model_path)
    plot_training_history(history)
    return model, history

# Evaluate model
def evaluate_model(model, test_dataset, class_names):
    y_pred = []
    y_true = test_dataset.classes
    predictions = model.predict(test_dataset)
    y_pred_classes = np.argmax(predictions, axis=1)
    cm = confusion_matrix(y_true, y_pred_classes)
    plot_confusion_matrix(cm, class_names)

def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(10, 7))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    fmt = 'd'
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="black")
    plt.show()

# Load images
def load_images(directory):
    image_files = glob.glob(os.path.join(directory, '*.*'))
    images = []
    for file in image_files:
        try:
            with Image.open(file) as img:
                img = img.resize((img_width, img_height))
                img_array = np.array(img) / 255.0
                if img_array.ndim == 3 and img_array.shape[2] == 3:
                    images.append((file, img_array))
        except Exception as e:
            print(f"Error loading image {file}: {e}")
    return images

# Splice image
def splice_image(file_path, n):
    img = Image.open(file_path)
    width, height = img.size
    cropped_images = []
    tile_width = width // n
    tile_height = height // n

    for i in range(n):
        for j in range(n):
            left = j * tile_width
            upper = i * tile_height
            right = left + tile_width
            lower = upper + tile_height
            cropped_img = img.crop((left, upper, right, lower))
            cropped_images.append((cropped_img, i * n + j + 1))
    return cropped_images


def preprocess_image(image):
    """
    Preprocess the input image by converting to grayscale and normalizing the contrast.
    
    Parameters:
    image (PIL.Image.Image): The input image.
    
    Returns:
    np.ndarray: The preprocessed grayscale image.
    """
    # Convert PIL image to OpenCV image format
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Normalize contrast
    normalized_image = cv2.equalizeHist(gray_image)
    return normalized_image

def segment_clouds(image, threshold_value=200):
    """
    Segment clouds in the preprocessed image using binary thresholding.
    
    Parameters:
    image (np.ndarray): The preprocessed grayscale image.
    threshold_value (int): The threshold value for binary segmentation (default: 200).
    
    Returns:
    np.ndarray: The binary mask for the clouds.
    """
    _, binary_mask = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    return binary_mask

def postprocess_mask(mask):
    """
    Clean up the binary mask using morphological operations.
    
    Parameters:
    mask (np.ndarray): The binary mask.
    
    Returns:
    np.ndarray: The cleaned binary mask.
    """
    kernel = np.ones((5, 5), np.uint8)
    cleaned_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_OPEN, kernel)
    return cleaned_mask

def calculate_cloud_coverage(mask):
    """
    Calculate the percentage of the image covered by clouds.
    
    Parameters:
    mask (np.ndarray): The cleaned binary mask.
    
    Returns:
    float: The percentage of the image covered by clouds.
    """
    total_pixels = mask.size
    cloud_pixels = cv2.countNonZero(mask)
    cloud_percentage = (cloud_pixels / total_pixels) * 100
    return cloud_percentage



def analyze_image_parts(image_path, n, threshold_percentage=30, threshold_value=200):
    """
    Analyze the image parts for cloud coverage and store parts that meet the criteria.
    
    Parameters:
    image_path (str): The path to the input image.
    n (int): Number of rows and columns to splice the image into.
    threshold_percentage (float): The threshold percentage for determining cloud coverage (default: 30).
    threshold_value (int): The threshold value for binary segmentation (default: 200).
    """
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_folder = os.path.join(os.path.dirname(image_path), "Filtered_Image")
    os.makedirs(output_folder, exist_ok=True)
    
    cropped_images = splice_image(image_path, n)
    
    for cropped_img, idx in cropped_images:
        preprocessed_image = preprocess_image(cropped_img)
        binary_mask = segment_clouds(preprocessed_image, threshold_value)
        cleaned_mask = postprocess_mask(binary_mask)
        cloud_percentage = calculate_cloud_coverage(cleaned_mask)
        
        status = "Accepted" if cloud_percentage < threshold_percentage else "Rejected"
        print(f"Part {idx} Cloud Coverage: {cloud_percentage:.2f}% - {status}")
        
        if cloud_percentage < threshold_percentage:  # Save only if cloud coverage is below the threshold
            output_name = f"{base_name}_Q{idx}.png"
            output_path = os.path.join(output_folder, output_name)
            cropped_img.save(output_path)


# Classify and save images

def classify_and_save_images(images, model, class_names, vegetation_classes, filtered_image_folder, n):
    if not os.path.exists(filtered_image_folder):
        os.makedirs(filtered_image_folder)

    for veg_class in vegetation_classes:
        class_folder = os.path.join(filtered_image_folder, veg_class)
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)

    for file_path, img_array in images:
        try:
            img_array_exp = np.expand_dims(img_array, axis=0)
            prediction = model.predict(img_array_exp)
            predicted_class = np.argmax(prediction, axis=1)[0]
            predicted_label = class_names[predicted_class]

            if predicted_label in vegetation_classes:
                # Splice the image if it's a vegetation class
                cropped_images = splice_image(file_path, n)
                
                for cropped_img, idx in cropped_images:
                    base_name = os.path.splitext(os.path.basename(file_path))[0]
                    new_filename = f"{base_name}_part{idx}.jpg"
                    save_path = os.path.join(filtered_image_folder, predicted_label, new_filename)

                    cropped_img.save(save_path)
                    print(f"Spliced image {file_path} classified as {predicted_label} and saved to {save_path}")
            else:
                print(f"Image {file_path} classified as {predicted_label} but not a vegetation class.")

        except Exception as e:
            print(f"Error processing image {file_path}: {e}")

def display_images(image_folder):
    """
    Display images from the specified folder.
    
    Parameters:
    image_folder (str): The path to the folder containing images.
    """
    images = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    num_images = len(images)
    
    if num_images == 0:
        print("No images found in the folder.")
        return
    
    plt.figure(figsize=(12, 6))
    for i, img_path in enumerate(images):
        img = Image.open(img_path)
        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.title(os.path.basename(img_path))
        plt.axis('off')
    plt.show()

def process_and_analyze_images(test_folder, n):
    """
    Process and analyze all images in the test folder.
    
    Parameters:
    test_folder (str): The path to the test folder containing images.
    n (int): Number of rows and columns to splice the images into.
    """
    for image_name in os.listdir(test_folder):
        if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(test_folder, image_name)
            analyze_image_parts(image_path, n)




# Load trained model
def load_trained_model(model_path='resnet50_model.h5'):
    model = load_model(model_path)
    print("Model loaded successfully.")
    return model



In [2]:

import glob
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import openeo

def load_data_from_openeo():
    # Connect to the OpenEO back-end
    connection = openeo.connect("https://openeo.dataspace.copernicus.eu")

    # Authenticate with the back-end
    connection.authenticate_oidc()

    # Load Sentinel-2 data collection
    s2_cube = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=("2022-05-01", "2022-05-30"),
        spatial_extent={
            "west": 3.20,
            "south": 51.18,
            "east": 3.25,
            "north": 51.21,
            "crs": "EPSG:4326",
        },
        bands=["B08", "B07", "B06", "B05", "B04", "B03", "B02"],
        max_cloud_cover=50,
    )
    
    # Compute the median for the temporal extent
    s2_median = s2_cube.reduce_dimension(dimension="t", reducer="median")
    
    # Download the results as an xarray dataset
    job = s2_median.execute_batch("output.nc", out_format="netCDF")
    job.download_results(".")

    ds = xr.open_dataset("output.nc")
    return ds

def load_data_from_directory(data_folder_path):
    pattern = "*.tiff"
    search_pattern = f"{data_folder_path}\\{pattern}"

    # Use glob to find files matching the pattern
    S_sentinel_bands = glob.glob(search_pattern)
    print("Files found:", S_sentinel_bands)

    if S_sentinel_bands:
        # Initialize a list to store band data and band names
        band_data = []
        band_names = []

        # Read each TIFF file and append the first band to the list
        for file_path in S_sentinel_bands:
            try:
                with rasterio.open(file_path) as ds:
                    band_data.append(ds.read(1))
                    band_names.append(ds.descriptions[0])
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

        # Stack all bands into a single array
        arr_st = np.stack(band_data)
        print("Shape of stacked bands:", arr_st.shape)

        # Create an xarray dataset
        ds = xr.Dataset({f"B{str(i+1).zfill(2)}": (["y", "x"], arr_st[i]) for i in range(len(arr_st))})
        return ds
    else:
        print("No files found.")
        return None

# Functions to calculate various indices
def calculate_evi(ds):
    nir = ds["B08"] * 0.0001
    red = ds["B04"] * 0.0001
    blue = ds["B02"] * 0.0001
    evi = 2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1))
    return evi

def calculate_lai(ds):
    evi = calculate_evi(ds)
    lai = (3.618 * evi - 0.118)
    return lai

def calculate_rvi(ds):
    nir = ds["B08"] * 0.0001
    red = ds["B04"] * 0.0001
    rvi = nir / red
    return rvi

def calculate_pssra(ds):
    nir = ds["B07"] * 0.0001
    red = ds["B04"] * 0.0001
    pssra = nir / red
    return pssra

def calculate_ndvi(ds):
    nir = ds["B08"] * 0.0001
    red = ds["B04"] * 0.0001
    ndvi = (nir - red) / (nir + red)
    return ndvi

def calculate_ndi45(ds):
    nir = ds["B05"] * 0.0001
    red = ds["B04"] * 0.0001
    ndi45 = (nir - red) / (nir + red)
    return ndi45

def calculate_gndvi(ds):
    nir = ds["B08"] * 0.0001
    green = ds["B03"] * 0.0001
    gndvi = (nir - green) / (nir + green)
    return gndvi

def calculate_mcari(ds):
    red2 = ds["B05"] * 0.0001
    red1 = ds["B04"] * 0.0001
    green = ds["B03"] * 0.0001
    mcari = ((red2 - red1) - 0.2 * (red2 - green)) * (red2 / red1)
    return mcari

def calculate_s2rep(ds):
    red1 = ds["B04"] * 0.0001
    nir = ds["B07"] * 0.0001
    red2 = ds["B05"] * 0.0001
    red3 = ds["B06"] * 0.0001
    s2rep = 705 + 35 * ((red1 + nir) / 2 - red2) / (red3 - red2)
    return s2rep

def calculate_ireci(ds):
    nir = ds["B07"] * 0.0001
    red1 = ds["B04"] * 0.0001
    red2 = ds["B05"] * 0.0001
    red3 = ds["B06"] * 0.0001
    ireci = (nir - red1) / (red2 / red3)
    return ireci

def calculate_savi(ds, L=0.428):
    nir = ds["B08"] * 0.0001
    red = ds["B04"] * 0.0001
    savi = (1 + L) * (nir - red) / (nir + red + L)
    return savi

def calculate_indices(ds):
    indices = {
        "NDVI": calculate_ndvi(ds),
        "SAVI": calculate_savi(ds),
        "RVI": calculate_rvi(ds),
        "PSSRa": calculate_pssra(ds),
        "NDI45": calculate_ndi45(ds),
        "GNDVI": calculate_gndvi(ds),
        "MCARI": calculate_mcari(ds),
        "S2REP": calculate_s2rep(ds),
        "IRECI": calculate_ireci(ds),
        "LAI": calculate_lai(ds),
        "EVI": calculate_evi(ds),
    }
    return indices

def plot_images(ds, index, title):
    fig, axes = plt.subplots(1, 2, figsize=(18, 6), dpi=90)

    red = ds["B04"] * 0.0001
    green = ds["B03"] * 0.0001
    blue = ds["B02"] * 0.0001
    rgb = np.stack([red, green, blue], axis=-1)
    rgb = np.clip(rgb / np.percentile(rgb, 99.5), 0, 1)

    axes[0].imshow(rgb)
    axes[0].set_title("RGB Composite")
    axes[0].set_xlabel("Pixel X")
    axes[0].set_ylabel("Pixel Y")

    cax2 = axes[1].imshow(index, cmap='RdYlGn', vmin=-1, vmax=1)
    axes[1].set_title(title)
    axes[1].set_xlabel("Pixel X")
    axes[1].set_ylabel("Pixel Y")
    
    cbar2 = plt.colorbar(cax2, ax=axes[1], orientation='vertical')
    cbar2.set_label(title)
    
    plt.tight_layout()
    plt.show()

def select_data_source():
    choice = input("Choose data source (1: OpenEO, 2: Local Directory): ").strip()

    if choice == '1':
        ds = load_data_from_openeo()
        print("Data loaded from OpenEO.")
    elif choice == '2':
        # Provide a hint for the directory path using a raw string
        default_path = r"C:\Users\abudh\Desktop\CropWatch\SunderabanData"
        print(f"Hint: The path to the data folder (e.g., {default_path})")
        
        # Prompt the user for the path, using the default path if none is provided
        data_folder_path = input(f"Enter the path to the data folder [{default_path}]: ").strip()
        if not data_folder_path:
            data_folder_path = default_path
        
        # Load data from the local directory
        ds = load_data_from_directory(data_folder_path)
        if ds is None:
            print("No data found or could not load data from the directory.")
            return
        
        print("Data loaded from local directory.")
    else:
        print("Invalid choice. Please select 1 or 2.")
        return

    # Calculate indices
    indices = calculate_indices(ds)
    
    # Plot results
    for index_name, index_data in indices.items():
        plot_images(ds, index_data, index_name)



In [ ]:
# Execute the workflow
train_dataset, test_dataset, class_names = prepare_data(dataset_url, img_height, img_width, batch_size, validation_split, rescale)
model, history = train_model(train_dataset, test_dataset, class_names, epochs=epoch)



Found 21600 images belonging to 10 classes.
Found 5400 images belonging to 10 classes.
Epoch 1/10


C:\Users\abudh\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


675/675 ━━━━━━━━━━━━━━━━━━━━ 1122s 1s/step - accuracy: 0.4607 - loss: 1.8914 - val_accuracy: 0.1980 - val_loss: 12.6360
Epoch 2/10
675/675 ━━━━━━━━━━━━━━━━━━━━ 3216s 5s/step - accuracy: 0.6753 - loss: 1.0207 - val_accuracy: 0.4565 - val_loss: 1.7815
Epoch 3/10
138/675 ━━━━━━━━━━━━━━━━━━━━ 8:59 1s/step - accuracy: 0.7403 - loss: 0.7700

In [ ]:
evaluate_model(model, test_dataset, class_names)


In [ ]:
images = load_images(filtered_image_folder)
classify_and_save_images(images, model, class_names, vegetation_classes, filtered_image_folder, n)

In [ ]:

# Analyze and display images
process_and_analyze_images(test_folder, n)
display_images(filtered_image_folder)

#Running Trained Model
# Main execution
model = load_trained_model(model_path)

#Classification of data

print("Loading and processing images...")
images = load_images(test_folder)
if images:
    classify_and_save_images(images, model, class_names, vegetation_classes, filtered_image_folder, n)
else:
    print("No images found or loading error.")
print("Image processing complete.")



# Directly call the select_data_source function; Local Disk or Copernicus API with desired location
select_data_source()
